In [1]:
import gym
import numpy as np
import random

eps = 1
gamma = 0.95
lr = 0.4
cleared = 0
epoch = 2000
use_noise = False

In [2]:
def getRandomMaxAction(q):
    index = (q==q.max()).nonzero()[0]
    return np.random.choice(index)

###### Q-Learning with learning-rate.

In [69]:
env = gym.make("FrozenLake-v0")
Q = np.zeros((obs_size,action_size))

action_size = env.action_space.n
obs_size = env.observation_space.n


for i in range(2000):
    obs = env.reset()
    while True:
        if use_noise:
            action = getRandomMaxAction(Q[obs]+np.random.randn(4)/(cleared+1))
        else:
            if random.random() < eps/(cleared+1):
                action = env.action_space.sample()
            else:
                action = getRandomMaxAction(Q[obs])
        next_obs, rew, done, etc = env.step(action)
        
        if next_obs != obs:
            Q[obs, action] = (1-lr) * Q[obs,action] + lr*(rew + gamma * Q[next_obs].max())
        
        if done:
            break
        obs = next_obs

    if rew == 1:
        cleared +=1
print("Cleared Rate = %.4f"%(cleared/epoch))

Cleared Rate = 0.5880


###### Q-Network

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

In [48]:
class Net(nn.Module):
    def __init__(self, in_size, hidden_size, out_size):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_size, out_size),
        )
    def forward(self, x):
        return self.net(x)

In [50]:
def getOneHot(idx, size):
    x = np.zeros((1,size))
    x[0,idx] = 1
    return torch.FloatTensor(x)

def init_normal(m):
        if type(m) == nn.Linear:
            nn.init.normal_(m.weight,0,0.01)

In [59]:
env = gym.make("FrozenLake-v0")

action_size = env.action_space.n
obs_size = env.observation_space.n
hidden_size = 64

QNet = Net(obs_size, hidden_size, action_size)
QNet.apply(init_normal)
loss = nn.MSELoss(reduction="sum")
opt = optim.Adam(QNet.parameters(), lr = 0.1)
cleared = 0
eps= 0.1

for i in range(epoch):
    obs = env.reset()
    while True:
        opt.zero_grad()
        Q = QNet(getOneHot(obs, obs_size))
        if random.random()< eps/(cleared+1):
            action = env.action_space.sample()
        else:
            action = Q.data.numpy().argmax()
        
        next_obs, rew, done, etc = env.step(action)
        
        if next_obs != obs:
            Y=Q.clone().detach()
            if done:
                Y[0, action] = rew
                break
            else:
                Y[0, action] = rew + gamma * QNet(getOneHot(next_obs, obs_size).detach()).data.numpy().max()
            loss_ = loss(Q, Y)
            loss_.backward(retain_graph=True)
            opt.step()    
        obs = next_obs
        
        if done:
            break
    if rew == 1:
        cleared +=1
    if i%(epoch/10) == 0:
        print("!",end="")
print()
print("Cleared Rate = %.4f"%(cleared/epoch))

!!!!!!!!!!
Cleared Rate = 0.0270


In [84]:
import gym
import collections

ENV_NAME = "FrozenLake-v0"
GAMMA = 0.9
ALPHA = 0.2
TEST_EPISODES = 20


class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.values = collections.defaultdict(float)

    def sample_env(self):
        action = self.env.action_space.sample()
        old_state = self.state
        new_state, reward, is_done, _ = self.env.step(action)
        self.state = self.env.reset() if is_done else new_state
        return (old_state, action, reward, new_state)

    def best_value_and_action(self, state):
        best_value, best_action = None, None
        for action in range(self.env.action_space.n):
            action_value = self.values[(state, action)]
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_value, best_action

    def value_update(self, s, a, r, next_s):
        best_v, _ = self.best_value_and_action(next_s)
        new_val = r + GAMMA * best_v
        old_val = self.values[(s, a)]
        self.values[(s, a)] = old_val * (1-ALPHA) + new_val * ALPHA

    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            _, action = self.best_value_and_action(state)
            new_state, reward, is_done, _ = env.step(action)
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward


if __name__ == "__main__":
    test_env = gym.make(ENV_NAME)
    agent = Agent()

    iter_no = 0
    best_reward = 0.0
    while True:
        iter_no += 1
        s, a, r, next_s = agent.sample_env()
        agent.value_update(s, a, r, next_s)

        reward = 0.0
        for _ in range(TEST_EPISODES):
            reward += agent.play_episode(test_env)
        reward /= TEST_EPISODES
        if reward > best_reward:
            print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
            best_reward = reward
        if reward > 0.80:
            print("Solved in %d iterations!" % iter_no)
            break

Best reward updated 0.000 -> 0.150
Best reward updated 0.150 -> 0.200
Best reward updated 0.200 -> 0.250
Best reward updated 0.250 -> 0.350
Best reward updated 0.350 -> 0.400
Best reward updated 0.400 -> 0.500
Best reward updated 0.500 -> 0.650
Best reward updated 0.650 -> 0.700
Best reward updated 0.700 -> 0.750
Best reward updated 0.750 -> 0.800
Best reward updated 0.800 -> 0.850
Solved in 6538 iterations!
